In [1]:
%load_ext autoreload
%autoreload 2

from PIL import Image
import napari
import numpy as np
import os

from scribbles_creator import *
from cellpose_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define where the images are located

In [2]:
folder_path = "./cellpose_train_imgs/"

## Create ground truth for pixel classification from image mask (instance segmentation)

In [ ]:
# for img_num in range(0, 540):
#     create_cellpose_gt(folder_path, img_num, save_res=True, show_res=False)

## Create scribble annotation from an image mask of the cell pose dataset

In [ ]:
# mode = "all"
# bins = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
# all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
# suff = all_suff[:5]

# percentages = {}
# for bin in bins:
#     percentages[bin] = []
#     for img_num in range(0, 10):
#         for s in suff:
#             print(f"IMG {img_num}: {bin}_{s}")
#             scribbles, perc_labelled = create_cellpose_scribble(folder_path, img_num, bin=bin, mode=mode, save_res=True, suff=s, show_res=False, print_steps=False)
#             percentages[bin].append(perc_labelled)
#             print("\n")

Same cell but for tests...

In [4]:
mode = "all"
bins = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = all_suff[:1]

percentages = {}
for bin in [0.5]:#bins:
    percentages[bin] = []
    for img_num in [200]:#, 50, 272, 351, 493]:#, 383, 419]:#range(0, 10):
        for s in suff:
            print(f"IMG {img_num}: {bin}_{s}")
            scribbles, perc_labelled = create_cellpose_scribble(folder_path, img_num, bin=bin, mode=mode, save_res=False, suff=s, show_res=True, print_steps=True, sq_scaling=False, scribble_width=1)
            percentages[bin].append(perc_labelled)
            print("\n")

IMG 200: 0.5_a
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 512, 512)
CLASS 1:
sk_max_pix: 374.93, sq_size: 21, sk_pix_range: (10, 42)
   prim_sk_squares pix: 368 = 0.16%
   sec_sk_squares pix: 374 = 0.17%
lines_max_pix: 374.93, line_pix_range: (10, 42)
-- Sampling lines: pix_in_sk 4304 indx_step 1 dist_to_edge 2
--    pix_in_line 2 added_pix 0
--    outside line_pix_range (10, 42)
--    pix_in_line 4 added_pix 0
--    outside line_pix_range (10, 42)
--    pix_in_line 64 added_pix 0
--    outside line_pix_range (10, 42)
--    pix_in_line 1 added_pix 0
--    outside line_pix_range (10, 42)
--    pix_in_line 5 added_pix 0
--    outside line_pix_range (10, 42)
--    pix_in_line 5 added_pix 0
--    outside line_pix_range (10, 42)
--    pix_in_line 48 added_pix 0
--    outside line_pix_range (10, 42)
--    pix_in_line 1 added_pix 0
--    outside line_pix_range (10, 42)
--    pix_in_line 65 added_pix 0
--    outside line_pix_range (10, 42)
--    pix_in_lin

Quick report of labelling percentages:

In [10]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

labelled percentage in bin 0.5: mean = 1.1459, std = 0.0000, range = 1.1459 - 1.1459


## Load and show scribbles and ground truth

In [ ]:
# Show the image, ground truth and the scribble annotation in a napari viewer
img_num = 0
mode = "all"
bin = 0.1
suff = "a"
img_data = get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=True, load_scribbles=True, mode=mode, bin=bin, suff=suff)
v = napari.Viewer()
v.add_image(img_data["img"], name="image")
v.add_labels(img_data["gt"], name="ground truth")
v.add_labels(img_data["scribbles"], name="scribbles")

## When we have performed segmentation in the Napari plugin, we can calculate its accuracy

In [ ]:
# segmentation = v.layers['segmentation'].data
# acc = np.round(100*np.sum(segmentation == ground_truth) / np.sum(ground_truth > 0), 2)
# print(f"Accuracy (pixels correctly segmented): {acc}%")

# TESTS

In [ ]:
from matplotlib import pyplot as plt

for i in range(1000):
    start = (225,25)
    row = [True] * 300
    mask = np.array([row] * 300)
    mask[50:255, 150:255] = False
    mask[50:100, 10:100] = False
    mask[200:250, 50:100] = False
    res = point_to_edge(start, mask)
# print(res)
# res_int = np.array(res, dtype=np.uint16)
# print(np.argwhere(res == False).shape)
# plt.imshow(res)

plt.imshow(np.logical_or(res, np.logical_not(mask)))
plt.colorbar()


In [ ]:
from matplotlib import pyplot as plt

row = [False] * 300
mask = np.array([row] * 300)
mask[50:73,140] = True
mask[52,143:160] = True
mask_changed = mask.copy()
plt.imshow(mask)

max_pix = 5
coord = np.where(mask)
# print(coord)

mask_changed[coord[0][max_pix:], coord[1][max_pix:]] = False
# for i in range(max_pix, len(coord[0])):
#     print(i)
#     print(coord[0][i], coord[1][i])
#     mask_changed[coord[0][i], coord[1][i]] = False

print("num pix:", np.sum(mask_changed))
plt.imshow(mask_changed)